# Biopython 3: SearchIO and Entrez

In this workshop, we will first take a quick look at SearchIO, Biopython's tool for unifying results from various sequence search tools, before exploring the Entrez API utility from NCBI.

## SearchIO

Biopython provides the [SearchIO package](https://biopython.org/docs/latest/api/Bio.SearchIO.html) for dealing with outputs from various sequence searching utilities, allowing them to be compared directly. Similar to SeqIO and AlignIO, it has capabilities for parsing, reading, writing, and indexing different search results, as well as converting between file types. 

### Reading and Parsing

Parsing and reading also work the same way as previously, where parse returns an iterator, and read simply returns the first item in the file. Note here that each item is a query, not an individual sequence. 

Let's start with an example of reading and XML result file from a BLAST search.

In [ ]:
# Parse a BLAST XML file

from Bio import SearchIO

handle = "searchio-data/blast.xml"

### Writing and Format Conversion

You can also write results files in various file formats. To do this, use the `Bio.SearchIO.write()` function. In the example below, we parse a file in the blast-xml format, and then write its contents to a new file in blast-tab tabular format. 

```
from Bio import SearchIO
qresults = SearchIO.parse(handle = 'sample.xml', format = 'blast-xml')
SearchIO.write(qresults, handle = 'sample.tab', format = 'blast-tab')
```

You can also use the `convert()` function to directly convert between file formats without reading/parsing and then writing. The supported formats are below, although not all of the pairs will work for conversion (see [documentation](https://biopython.org/docs/latest/api/Bio.SearchIO.html#Bio.SearchIO.convert) for details).

'blast-tab', 'blast-xml', 'blat-psl', 'hmmer3-tab', 'hmmscan3-domtab', 'hmmsearch3-domtab', 'phmmer3-domtab"

The function will return a tuple of four values: the number of QueryResult, Hit, HSP, and HSPFragment objects it writes to the output file.

In [ ]:
in_file = "searchio-data/blast.xml"
out_file = "searchio-data/blast_tab.tab"

## Entrez

The Entrez system is a collection of [NCBI databases](https://www.ncbi.nlm.nih.gov/guide/all/) ranging from literature to sequences, along with a text search tool for exploring them. You can [search online](https://www.ncbi.nlm.nih.gov/search/) via browser, or through the NCBI's E-utilities API. 

### Entrez Rules and Etiquette

The good news is that Biopython takes care of a lot of the Entrez etiquette for you (limiting the number of queries per second, etc.). However, there are a few things that you must do yourself:

1. Always define your Entrez email address (your code will not run without this parameter)
2. Specify your tool if you are using Biopython within some larger software package (define `Entrez.tool`)
3. Use the session history for large queries
4. For large series of requests (>100) try to do this outside of US peak hours

### UIDs

Every entry in a NCBI database has a UID (unique identifier). This UID will vary depending on the database. For example, PubMed uses PMID while protein records use GI numbers. 

| Entrez Database    | UID common name | E-utility Database Name |
|--------------------|-----------------|-------------------------|
| BioProject         | BioProject ID  | bioproject              |
| BioSample          | BioSample ID   | biosample               |
| Books              | Book ID        | books                   |
| Conserved Domains  | PSSM-ID        | cdd                     |
| dbGaP              | dbGaP ID       | gap                     |
| dbVar              | dbVar ID       | dbvar                   |
| Gene               | Gene ID        | gene                    |
| Genome             | Genome ID      | genome                  |
| GEO Datasets       | GDS ID         | gds                     |
| GEO Profiles       | GEO ID         | geoprofiles             |
| HomoloGene         | HomoloGene ID  | homologene              |
| MeSH               | MeSH ID        | mesh                    |
| NCBI C++ Toolkit   | Toolkit ID     | toolkit                 |
| NLM Catalog        | NLM Catalog ID | nlmcatalog              |
| Nucleotide         | GI number      | nuccore                 |
| PopSet             | PopSet ID      | popset                  |
| Probe              | Probe ID       | probe                   |
| Protein            | GI number      | protein                 |
| Protein Clusters   | Protein Cluster ID | proteinclusters      |
| PubChem BioAssay   | AID            | pcassay                 |
| PubChem Compound   | CID            | pccompound              |
| PubChem Substance  | SID            | pcsubstance             |
| PubMed             | PMID           | pubmed                  |
| PubMed Central     | PMCID          | pmc                     |
| SNP                | rs number      | snp                     |
| SRA                | SRA ID         | sra                     |
| Structure          | MMDB-ID        | structure               |
| Taxonomy           | TaxID          | taxonomy                |

**Accession.Version vs GI Number**

Sequences will have two parallel identifiers given by the NCBI: the GI number and the Accession version. For a full disambiguation, see this [link](https://www.ncbi.nlm.nih.gov/genbank/sequenceids/). 

### E-Utilities on the Unix Command Line

While Biopython gives us nice tools for using the E-utilities withi Python code, they are also available as [command line tools](https://www.ncbi.nlm.nih.gov/books/NBK179288/)

### Core Concepts of the E-utilities

**What are the E-utilities?**

There are nine total E-utilities which perform different tasks with respect to the NCBI databases (from [A General Introduction to the E-utilities](https://www.ncbi.nlm.nih.gov/books/NBK25497/)).

1. EInfo (database statistics) eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi

    Provides the number of records indexed in each field of a given database, the date of the last update of the database, and the available links from the database to other Entrez databases.

2. ESearch (text searches) eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi

    Responds to a text query with the list of matching UIDs in a given database (for later use in ESummary, EFetch or ELink), along with the term translations of the query.

3. EPost (UID uploads) eutils.ncbi.nlm.nih.gov/entrez/eutils/epost.fcgi

    Accepts a list of UIDs from a given database, stores the set on the History Server, and responds with a query key and web environment for the uploaded dataset.

4. ESummary (document summary downloads) eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi

    Responds to a list of UIDs from a given database with the corresponding document summaries.

5. EFetch (data record downloads) eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi

    Responds to a list of UIDs in a given database with the corresponding data records in a specified format.

6. ELink (Entrez links) eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi

    Responds to a list of UIDs in a given database with either a list of related UIDs (and relevancy scores) in the same database or a list of linked UIDs in another Entrez database; checks for the existence of a specified link from a list of one or more UIDs; creates a hyperlink to the primary LinkOut provider for a specific UID and database, or lists LinkOut URLs and attributes for multiple UIDs.

7. EGQuery (global query) eutils.ncbi.nlm.nih.gov/entrez/eutils/egquery.fcgi

    Responds to a text query with the number of records matching the query in each Entrez database.

8. ESpell (spelling suggestions) eutils.ncbi.nlm.nih.gov/entrez/eutils/espell.fcgi

    Retrieves spelling suggestions for a text query in a given database.

9. ECitMatch (batch citation searching in PubMed) eutils.ncbi.nlm.nih.gov/entrez/eutils/ecitmatch.cgi

    Retrieves PubMed IDs (PMIDs) corresponding to a set of input citation strings.

## Bio.Entrez Module

Biopython gives us a module that allows us to perform these functions inside of a Python script. What it essentially does is to take our arguments, generate the Entrez E-utility URL, send to the NCBI server, and deal with the returned information. The documentation can be found [here](https://biopython.org/docs/latest/api/Bio.Entrez.html). As well as the 9 core functions, the module provides functions for reading and parsing the results.

### An Example with EInfo

Here we will use the first function, EInfo, to get information on some NCBI databases. The basic routine is as follows:

0. Give Entrez your email address.
1. Define your handle as the output of the `Entrez.einfo()` function.
2. Create a record by "reading" the handle using the `Entrez.read()` function. This record will be a nest of Python dictionaries - explore the list by looking at the keys.
3. Extract the information you need by calling the specific keys as needed.

When given no arguments, `einfo()` returns a list of all valid Entrez databases. You can also specify a database by giving it a `db` (database) variable. Then, it will return the information related to that specific database. There are a couple of other inputs for specifying the output format (JSON is available as well as the default XML) and a version input (which is for the very interested user only - check the documentation for details).

First, let's get all of the database information.

Next, we can find out more information about the SRA (Sequence Read Archive) database.

### A quick aside: finding the DTD file

Let's do our Einfo search again, but instead of using the `Entrez.read()` function, which translates the XML returned by EInfo to Python dictionaries and lists, read and print it as an XML file so we can examine it more closely.

In [ ]:
out_handle = open("entrez-data/einfo.xml", "w")

Take a look at the `einfo.xml` results. Note that before the beginning of the results, denoted by `<eInfoResult>`, the header contains information about the file encoding as well as the DTD file used to define the XML data returned. Click on this link and download/open the file. If you ever receive a warning about your DTD file when you use `Entrez.read()`, you will see a link for the updated DTD file in the warning. Biopython will automatically access this new DTD online and continue as usual, but if you want better performance you can update the DTD on your machine (see the Biopython Tutorial document Ch. 9 for complete instructions).

## An Example with Genbank Data

Here we will use several of the E-utilities to search for genomic data of the RPL16 gene of a prickly pear (opuntia) in the [NCBI Nucleotide Database](https://www.ncbi.nlm.nih.gov/nucleotide/). This will occur in several stages:

1. Use EGQuery to find the number of hits for your search terms in the "nuccore" database.
2. Use ESearch to get the UIDs searching in "nuccore" with the desired search terms
3. Use EFetch to get the data associated  with those UIDs
4. Parse this sequence data and write it to a .gb file
5. Check your work by parsing the .gb file

In this example, we won't use the History server for simplicity - we will revoisit this example later to show how to use the History server properly. Let's begin by finding the number of results in the "nuccore" database.


Now that we know how many results are in the Nucleotide database, we can use ESearch to retrieve the list of UIDs. 

Finally, we can use EFetch to get the formatted data, and SeqIO to write them to a data file.

To check our work, let's parse the new data file we have created and print the first 10 base pairs in each sequence, as well as the sequence ID. 

In [ ]:
records = SeqIO.parse(handle = "entrez-data/opuntia.gb", format = "genbank")

## Using the History Feature

When we create pipelines of different E-utilities, the best practice is to store the intermediate information on the NCBI History server, rather than on our own devices. To add an item to the History server, we can use the EPost E-utility, or we can set the `usehistory = "y"` parameter when using ESearch. Each item in the History server will have two identifying parameters: the WebEnv (cookies string) and an integer Query Key. By invoking these identifiers, we can access our information directly from the server, rather than having to use our own device's memory. Note that we can also combine different records in the History server, which will share a WebEnv but have different Query Keys.

In the example below, we will run an ESearch on PubMed, setting `usehistory = "y"` so that the results are posted on the server. Then, we will pull the results from the server using EFetch to retrieve the citation information about these items. Without the history, we would have had to define the list of UIDs as a variable in our code, and then call the EFetch function on that list of UIDs. 

In the code block below, we will run the initial search.

In [ ]:


handle = Entrez.esearch(db = "pubmed", term = "Opuntia[ORGN]", reldate = 365, datetype = "pdat", usehistory = "y")


Now that we have performed the search, the results are saved in our History. Now, we can use the EFetch function to refer to these results and get the full citations. We will also download them in batches of 10. 

We now have a MEDLINE formatted file with all 128 citations. We can easily add this to any reference management software. Alternatively, we can use the [`Bio.Medline`](https://biopython.org/docs/latest/api/Bio.Medline.html) module to parse this file and extract additional information. The Bio.Medline package has familiar functions that read and parse MEDLINE formatted text files into Python dictionaries. Below I will show a quick example of this.


### Revisiting our GenBank Search

With this in mind, let's also revisit our Opuntia sequence search and do things the "right" way using the History server. First, let's redo our search but this time using `usehistory = "y"`. Note that we no longer have to deal with the awkward problem of "retmax" since the results are being stored on the server, and we are not actually retrieving the list of UIDs here! All we need is the "Count", which is the total number of results rather than the number of UIDs returned. 

Now that we have the search saved in the History, we can download the results in batches (best practice). This time, let's write them to a file in FASTA format. For all of the available retrieval modes (defined by the `retmode` variable), see this [table](https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/?report=objectonly).

In [ ]:
batch_size = 10
out_handle = open("entrez-data/opuntia.fasta", "w")

for start in range(0, count, batch_size):
    end = min(count, start + batch_size)
    print("Downloading records {} through {}".format(start + 1, end))


Now, let's parse the data and see that it is the same as what we got before!

## An Example with ELink

Let's use a different E-utility this time: ELink. ELink allows you to search through NCBI databases for related items. This can be used to find associated publictions by terms, data, or other publications, find related datasets, and more. The full documentation can be found in the Entrez Help document as well as in the Biopython Tutorial (Section 9.7).

ESearch has 3 required arguments: 

- `db`, the database to look for connections in
- `dbfrom`, the datbase of the original item we want to find connections to
- `id`, the list of UIDs in `dbfrom`

In the example below, we will search for a term in the taxonomy database using ESearch, and then ue the UID found as an input to ELink to find relted publications in PubMed. 

Now, we will use ELink to find related items in PubMed.

Finally, let's print out some of the PubMed results and extract some of the information, such as Journals and DOIs. 

## Putting It All Together & Final Thoughts

### Dealing with Large Data: Reading vs Parsing

As we have seen throughout the Biopython module, when dealing with large data files it is better to parse them rather than reading them. Rather than creating one Python object for your entire data file (in this case an XML file), the `Entrez.parse()` function allows you to read XML records one-by-one. In this workshop, we have not be dealing with large data files and will simply use the `Entrez.read()` function. Note that one result from an E-utility can't be parsed because it counts as one "item". This is intended for other types of XML data from NCBI.

### Building Python Functions with E-utilities

All of the code we wrote previously could be easily worked into Python functions. For example, you could define a function that searches PubMed for a given search term and then generates a citation file all in one step (or a similar scheme for retrieving data files).


## Useful Links and Documentation

* [Bio.SearchIO Documentation](https://biopython.org/docs/latest/api/Bio.SearchIO.html)
* [Bio.Entrez Documentation](https://biopython.org/docs/latest/api/Bio.Entrez.html)
* [Entrez Help](https://www.ncbi.nlm.nih.gov/books/NBK3837/)
* [Table of Entrez Databases and UIDs](https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/)
* [Values of retmode and rettype for EFetch](https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/?report=objectonly)
* [Table of ELinks](https://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html)

## Credits and Inspiration

* [Biopython Tutorial and Cookbook](https://biopython.org/DIST/docs/tutorial/Tutorial.html)
